In [20]:
#tabular data imports :
import pandas as pd
import numpy as np
import env
from env import username, password, host
from pydataset import data

# visualization imports:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
# success metrics from earlier in the week: mean squared error and r^2 explained variance
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
#stats
from scipy.stats import pearsonr, spearmanr
from scipy.stats import shapiro

import wrangle as w
import warnings
warnings.filterwarnings("ignore")
import os
directory = os.getcwd()

In [34]:
df = pd.read_csv('punching_shear_clean.csv')
df.head(3)

,Shape,b1 (mm),d1 (mm),davg (mm),ravg,b* (mm),b*/davg,fc (MPa),fy (MPa),Pu (kN)
0,C,51,NaN,38.100,0.020138,39.898227,1.047198,13.145888,550,24
1,S,76,NaN,29.972,0.017700,76.200000,2.542373,26.332005,396,31
2,C,102,NaN,38.100,0.020138,79.796453,2.094395,13.145888,550,33


# CURRENT DATA AND UNITS
## - Name -- Data Type -- Measurement -- Description

#### - Cement (component 1) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Blast Furnace Slag (component 2) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Fly Ash (component 3) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Water (component 4) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Superplasticizer (component 5) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Coarse Aggregate (component 6) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Fine Aggregate (component 7) -- quantitative -- kg per $m^3$ mixture -- Input Variable
#### - Age -- quantitative -- Day (1~365) -- Input Variable
#### - Concrete compressive strength -- quantitative -- MPa -- Output Variable

## Will be converting the units to US Imperial units

In [35]:
# Conversion factors
mm_to_inches = 1 / 25.4
MPa_to_psi = 145.038
kN_to_lbs = 224.809

def convert_units(df):
    # Apply conversions
    df['b1 (in)'] = df['b1 (mm)'] * mm_to_inches
    df['d1 (in)'] = df['d1 (mm)'] * mm_to_inches
    df['d avg (in)'] = df['davg (mm)'] * mm_to_inches
    df['b* (in)'] = df['b* (mm)'] * mm_to_inches
    df['fc (psi)'] = df['fc (MPa)'] * MPa_to_psi
    df['fy (psi)'] = df['fy (MPa)'] * MPa_to_psi
    df['pu (lbs)'] = df['Pu (kN)'] * kN_to_lbs

    # Lowercase all column names and remove spaces
    df.columns = [column.lower().replace(' ', '') for column in df.columns]
    
    # Add 'sample' column as the first column
    df.insert(0, 'sample', range(len(df)))
    
    # Set 'sample' column as the new index
    df.set_index('sample', inplace=True)
    
    # Drop the original metric columns if you no longer need them
    metric_columns = ['b1(mm)', 'd1(mm)', 'davg(mm)', 'b*(mm)', 'fc(MPa)', 'fy(MPa)', 'Pu(kN)']
    df = df.drop(columns=metric_columns, errors='ignore')
    
    return df

In [36]:
df = convert_units(df)
df.head(3)

,shape,ravg,b*/davg,fc(mpa),fy(mpa),pu(kn),b1(in),d1(in),davg(in),b*(in),fc(psi),fy(psi),pu(lbs)
sample,,,,,,,,,,,,,
0,C,0.020138,1.047198,13.145888,550,24,2.007874,NaN,1.50,1.570796,1906.653366,79770.900,5395.416
1,S,0.017700,2.542373,26.332005,396,31,2.992126,NaN,1.18,3.000000,3819.141341,57435.048,6969.079
2,C,0.020138,2.094395,13.145888,550,33,4.015748,NaN,1.50,3.141593,1906.653366,79770.900,7418.697


In [12]:
# information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 416
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   shape      417 non-null    object 
 1   ravg       417 non-null    float64
 2   b*/davg    417 non-null    float64
 3   fc (mpa)   417 non-null    float64
 4   fy (mpa)   417 non-null    int64  
 5   pu (kn)    417 non-null    int64  
 6   b1 (in)    417 non-null    float64
 7   d1 (in)    22 non-null     float64
 8   davg (in)  417 non-null    float64
 9   b* (in)    417 non-null    float64
 10  fc (psi)   417 non-null    float64
 11  fy (psi)   417 non-null    float64
 12  pu (lbs)   417 non-null    float64
dtypes: float64(10), int64(2), object(1)
memory usage: 45.6+ KB


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ravg,417.0,0.000457,0.000289,0.000000,0.000285,0.000417,0.000589,1.972638e-03
b*/davg,417.0,1.853056,0.941617,0.306909,1.200000,1.735043,2.222222,8.000000e+00
fc (mpa),417.0,32.237630,17.953680,8.662000,22.135000,27.700000,35.340000,1.187025e+02
fy (mpa),417.0,461.633094,118.227971,250.000000,359.000000,462.000000,530.000000,7.490000e+02
pu (kn),417.0,375.414868,436.956517,24.000000,166.000000,277.000000,404.000000,4.915000e+03
b1 (in),417.0,7.581903,4.155755,2.007874,4.724409,6.692913,9.842520,3.547244e+01
d1 (in),22.0,14.906943,5.339582,5.984252,10.836614,14.173228,18.897638,2.362205e+01
davg (in),417.0,4.354870,2.526022,1.180000,3.031496,4.212598,4.785756,2.631890e+01
b* (in),417.0,7.195284,3.717553,1.570796,4.328966,6.833582,9.276356,2.785999e+01
fc (psi),417.0,4675.681411,2603.965894,1256.319156,3210.416130,4017.552600,5125.642920,1.721637e+04


In [14]:
# checking for null values
df.isnull().sum()

shape          0
ravg           0
b*/davg        0
fc (mpa)       0
fy (mpa)       0
pu (kn)        0
b1 (in)        0
d1 (in)      395
davg (in)      0
b* (in)        0
fc (psi)       0
fy (psi)       0
pu (lbs)       0
dtype: int64

In [15]:
# checking for possible categorical features
df.nunique()

shape          3
ravg         186
b*/davg      181
fc (mpa)     300
fy (mpa)     123
pu (kn)      288
b1 (in)       56
d1 (in)       14
davg (in)    101
b* (in)       83
fc (psi)     300
fy (psi)     123
pu (lbs)     288
dtype: int64

In [16]:
# distribution of age samples
df.pu.value_counts()

AttributeError: 'DataFrame' object has no attribute 'age'

In [13]:
# List of components
components = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg']

# Sum the values across the specified columns and create the new column
df['total_lbs_per_yd^3'] = df[list(components)].sum(axis=1)
df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total_lbs_per_yd^3
sample,,,,,,,,,,
0,311.512806,467.379440,0.000000,448.640170,0.000000,2142.449716,1650.158070,28,4335.18582,5020.140202
1,372.360318,93.034964,274.034266,348.991346,23.809896,2382.753296,1755.318444,14,3409.84338,5250.302530
2,551.155000,0.000000,210.982134,413.145788,12.125410,2109.600878,1898.618744,28,4238.01036,5195.627954


In [14]:
# edited the function so im reruning it once more
df = w.prepare_data3(concrete)
df.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total_lbs_per_yd^3
sample,,,,,,,,,,
0,686.767362,1030.394061,0.000000,989.081092,0.000000,4723.287493,3637.971484,28,628766.680961,11068.0
1,820.913004,205.106742,604.141424,769.393301,52.491773,5253.065571,3869.810148,14,494556.864148,11575.0
2,1215.087336,0.000000,465.135432,910.829467,26.731921,4650.868288,4185.732855,28,614672.546594,11454.0


### will be adding a column that takes columns and represents them as percent in relation to total weight.

In [15]:
# List of components
components = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg']

# Calculate the percentage of each component relative to the total_lbs_per_yd^3
for component in components:
    df[component + '_pct'] = round((df[component] / df['total_lbs_per_yd^3']) * 100,2)

# Display the first few rows to verify the new columns
df.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total_lbs_per_yd^3,cement_pct,slag_pct,ash_pct,water_pct,superplastic_pct,coarseagg_pct,fineagg_pct
sample,,,,,,,,,,,,,,,,,
0,686.767362,1030.394061,0.000000,989.081092,0.000000,4723.287493,3637.971484,28,628766.680961,11068.0,6.20,9.31,0.00,8.94,0.00,42.68,32.87
1,820.913004,205.106742,604.141424,769.393301,52.491773,5253.065571,3869.810148,14,494556.864148,11575.0,7.09,1.77,5.22,6.65,0.45,45.38,33.43
2,1215.087336,0.000000,465.135432,910.829467,26.731921,4650.868288,4185.732855,28,614672.546594,11454.0,10.61,0.00,4.06,7.95,0.23,40.60,36.54
3,1292.852926,554.079825,0.000000,1108.159651,0.000000,4529.845589,3256.434061,28,964501.583207,10741.0,12.04,5.16,0.00,10.32,0.00,42.17,30.32
4,752.382079,891.388070,0.000000,939.505528,44.229179,5090.729903,3386.205388,28,384748.832211,11104.0,6.78,8.03,0.00,8.46,0.40,45.85,30.50
